## URL 확인

In [ ]:
https://search.naver.com/search.naver?where=news
    &query=서울시장%20핵심공약
    # &sm=tab_opt
    # &sort=0 관련도순이 default값이라 삭제
    # &photo=0
    # &field=0
    # &reporter_article=
    # &pd=3
    # &ds=2021.03.01
    # &de=2021.03.25
    # &docid=
    &nso=so%3Ar%2Cp%3Afrom20210301to20210325%2Ca%3Aall
    # &mynews=0
    # &refresh_start=0
    # &related=0

In [ ]:
https://search.naver.com/search.naver?where=news&query=서울시장%20핵심공약&nso=so%3Ar%2Cp%3Afrom20210301to20210325%2Ca%3Aall

## Scraping

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import time
import re

In [2]:
max_page = 10
start_points = []

for point in range(1, max_page*10+1, 10):
    start_points.append(str(point))

start_points

['1', '11', '21', '31', '41', '51', '61', '71', '81', '91']

In [3]:
titles = []
dates = []
articles = []
article_urls = []
press_companies = []

query = '서울시장 핵심공약'

start_date = '20210301'
end_date = '20210324'

In [4]:
current_call = 1
last_call = (max_page - 1) * 10 + 1

# 크롤링 시작
while current_call <= last_call :
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query \
            + "&nso=so%3Ar%2Cp%3Afrom"+ start_date \
            + "to" + end_date \
            + "%2Ca%3Aall" + "&start=" + str(current_call)
    
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        # 1) 네이버 뉴스만 추려내기
        if urls["href"].startswith("https://news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            # Header 정보 추가
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            # 2) 기사 제목 
            title = source_news.find('h3', {'id' : 'articleTitle'}).get_text()
            titles.append(title)
            print('Processing article : {}'.format(title))

            # 3) 기사 날짜
            date = source_news.find('span', {'class' : 't11'}).get_text()
            dates.append(date)

            # 4) 기사 본문
            article = source_news.find('div', {'id' : 'articleBodyContents'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()
            articles.append(article)

            # 5) 기사 URL 
            article_urls.append(url)

            # 6) 기사 발행 언론사
            press_company = source_news.find('address', {'class' : 'address_cp'}).find('a').get_text()
            press_companies.append(press_company)
            
        except:
            print("error : {}".format(url))
            
    time.sleep(5)
    current_call += 10

        
# 전체 데이터를 DataFrame에 모으고 엑셀로 저장
article_df = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
article_df.head()

Processing article : 서울시장 野 단일 후보 오세훈 확정...4·7 재보선 대진표 확정
Processing article : '박영선표' 청년 공약…"국회를 청년과 시민 품으로"
Processing article : 핵심은 '부동산 공급'…박 "공공 주도" vs 오 "민간 재건축"
Processing article : '경쟁력'은 끝났다…오세훈 vs 안철수, 단일화 핵심은 '정권 교체'
Processing article : 박영선-오세훈 누가 시장돼도 재개발-재건축 허용할 듯
Processing article : 박영선-이재명 '맞손'…재난위로금 공약 띄우기
Processing article : 박영선 "오세훈, MB 연상…안철수는 핵심 못 짚어"
Processing article : "박영선 공약에 열렬한 지지" 친이재명계 잇따른 선언 왜
Processing article : ‘공통 공약’ 재개발·재건축…수혜자 따로 있었다
Processing article : 박영선, '서울시 재난지원금' 공약 우선 보류...야권 단일화 기싸움 격화
Processing article : 서울시장 후보들 성평등 공약, 빈약하거나 핵심 못 짚어
Processing article : 페미니스트 시장이 필요합니다, 2021년이잖아요
Processing article : ‘퍼주기’식 금융지원 치중..4·7 재보선 공약 ‘표’퓰리즘 될라
Processing article : 야권 겨냥 화력 모으는 박영선… 민생공약 발표 이어가
Processing article : [주진우 라이브] 최지은 민주당 대변인 “오세훈 시장되면 공직자 신뢰 가장 큰 걸림돌 될 것…박형준은 시장돼도 검찰조사 받다 임기 끝날 것” vs 김병민 국민의힘 비대위원 “오세훈이 MB 황태자? 지고 있는 박영선의
Processing article : [젠더마이크] 서울시장 선거에 '청년여성'은 없다
Processing article : 이광재 사무총장 "서울시장 부동산 空약 난립…체계적 계획 내놓아야" [

,Title,Date,Article,URL,PressCompany
0,서울시장 野 단일 후보 오세훈 확정...4·7 재보선 대진표 확정,2021.03.23. 오후 12:12,"■ 진행 : 김정아 앵커 ■ 출연 : 김형준 / 명지대 교수, 배종호 / 세한대 교...",https://news.naver.com/main/read.nhn?mode=LSD&...,YTN
1,"'박영선표' 청년 공약…""국회를 청년과 시민 품으로""",2021.03.23. 오후 3:42,사진=연합뉴스 서울 영등포구 여의도동 1번지. 여의도 총 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,JTBC
2,"핵심은 '부동산 공급'…박 ""공공 주도"" vs 오 ""민간 재건축""",2021.03.24. 오전 7:54,"[앵커]서로에 대한 공격 포인트도 부동산이지만, 박영선·오세훈 두 후보가 경쟁하는 ...",https://news.naver.com/main/read.nhn?mode=LSD&...,JTBC
3,"'경쟁력'은 끝났다…오세훈 vs 안철수, 단일화 핵심은 '정권 교체'",2021.03.23. 오전 12:01,22일 서울시장 보궐선거 야권 단일화를 위한 여론조사가 시작된 가운데 야권 정계개편...,https://news.naver.com/main/read.nhn?mode=LSD&...,더팩트
4,박영선-오세훈 누가 시장돼도 재개발-재건축 허용할 듯,2021.03.23. 오전 11:54,서울시 시장 후보로 격돌할 박영선-오세훈 후보 동아일보 DB 보름 앞으로 다가온 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,donga.com
